In [ ]:
import pandas as pd
pd.set_option('mode.chained_assignment', None)
import numpy as np
from IPython.display import HTML
import IPython.core.display as di
import os, sys
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
homedir=os.getcwd()
from plotly import express as px 


In [ ]:
# Reading in the naming convention and colour code file
colorcode=pd.read_csv("tech_codes.csv",sep=',',encoding = "ISO-8859-1")
homedir=os.getcwd()

In [ ]:
#please change this to your directory of the results. 
#You HAVE TO name the result folders as they are named below. like "REF", "WAT"

res_folder=['C:/git/teaching-CLEWs_visualization/results/REF',
            'C:/git/teaching-CLEWs_visualization/results/CLM',
            'C:/git/teaching-CLEWs_visualization/results/COM',
            'C:/git/teaching-CLEWs_visualization/results/ENR',
            'C:/git/teaching-CLEWs_visualization/results/LND',
            'C:/git/teaching-CLEWs_visualization/results/WAT']

In [ ]:
# Making dictionaries linking the technology names to actual names and colour codes
colorcode1=colorcode.drop('colour',axis=1)
colorcode2=colorcode.drop('tech_code',axis=1)
det_col=dict([(a,b) for a,b in zip(colorcode1.tech_code,colorcode1.tech_name)])
color_dict=dict([(a,b) for a,b in zip(colorcode2.tech_name,colorcode2.colour)])

In [ ]:
#Central plotting function
def df_plot(df,y_title,p_title):
    if len(df.columns)==1:
        print('There are no values for the result variable that you want to plot')
    else:
        #fig1 = plt.figure()
        df=df.set_index('YEAR')         
        fig1 = df.plot(kind="bar", stacked=True,grid=False, figsize=(15,5), color=[color_dict[x] for x in df.columns if x != 'YEAR'])
        fig1.set_title(p_title,color='black',fontsize=15)
        fig1.set_ylabel(y_title,color='black',fontsize=15)
        fig1.set_xlabel('Year',color='black',fontsize=15)
        fig1.legend(bbox_to_anchor=(1.0, 0.75))
        fig1.plot()
        #Hash the following line if you want to see inline graphs
        #plt.close()   
        #fig1.figure.savefig('{}.jpeg'.format(p_title),bbox_inches='tight')
        fig1.figure.savefig(homedir +'/{}.pdf'.format(p_title),bbox_inches='tight')
        #df.to_csv(os.path.join(homedir,p_title+".csv"))
        #return iplot(fig)

In [ ]:
# Creating list with LAnd technology names needed for Land Graphs
land_tech=['AGRPST001','LNDBLT001', 'LNDCRP001I', 'LNDCRP001R', 'LNDCRP002I',
       'LNDCRP002R', 'LNDFOR001', 'LNDOTH001']
crop_tech=['LNDCRP001I', 'LNDCRP001R', 'LNDCRP002I',
       'LNDCRP002R']
pwr_tech=['PWRBIO001', 'PWRDSL001', 'PWRGAS001', 'PWRHYD001', 'PWRSOL001', 'PWRWND001']
wat_tech=['WATTRN001','WATELC001','WATAGR001']

In [ ]:
#Remove the old pf and png files formt the results folder
for f in res_folder:
    homedir=f
    
    for l in os.listdir(homedir):
        if not l.endswith(".png") | l.endswith(".pdf") :
            continue
        os.remove(os.path.join(homedir, l))
        

In [ ]:
# for loop to loop between the different resuls folders. The folders must be named just with the scenario name
for f in res_folder:
    #changing the home directory
    homedir=f
    scen=homedir[-3:]
    #print(homedir)
    tot_ann_cap=pd.read_csv(homedir+'/TotalCapacityAnnual.csv')
    tot_ann_cap.drop('REGION',axis=1,inplace=True)
    tot_ann_cap=tot_ann_cap.set_index('YEAR')
    prod_tech_ann=pd.read_csv(homedir+'/ProductionByTechnologyAnnual.csv')
    prod_tech_ann.drop('REGION',axis=1,inplace=True)
    prod_tech_ann=prod_tech_ann.set_index('YEAR')
    useby_tech_ann=pd.read_csv(homedir+'/UseByTechnology.csv')
    useby_tech_ann.drop('REGION',axis=1,inplace=True)
    useby_tech_ann=useby_tech_ann.set_index('YEAR')
    emissions=pd.read_csv(homedir+'/AnnualEmissions.csv')
    emissions.drop('REGION',axis=1,inplace=True)
    emissions=emissions.set_index('YEAR')
    emis_by_tech=pd.read_csv(homedir+'/AnnualTechnologyEmission.csv')
    emis_by_tech.drop('REGION',axis=1,inplace=True)
    emis_by_tech=emis_by_tech.set_index('YEAR')
    ##
    # Land Dataframes
    tot_land_cap= prod_tech_ann[prod_tech_ann['COMMODITY']=='LND001'].pivot_table(index='YEAR', 
                                               columns='TECHNOLOGY',
                                               values='VALUE', 
                                               aggfunc='sum').reset_index().rename_axis(None, axis=1).rename(columns=det_col).fillna(0)
    tot_land_cap=tot_land_cap[tot_land_cap['YEAR'].between(2020,2050)]


    tot_crop_cap= prod_tech_ann[(prod_tech_ann['COMMODITY']=='LND001') & (prod_tech_ann['TECHNOLOGY'].isin(crop_tech))].pivot_table(index='YEAR', 
                                                   columns='TECHNOLOGY',
                                                   values='VALUE', 
                                                   aggfunc='sum').reset_index().rename_axis(None, axis=1).rename(columns=det_col).fillna(0)
    tot_crop_cap=tot_crop_cap[tot_crop_cap['YEAR'].between(2020,2050)]

    tot_crop_prod=prod_tech_ann[(prod_tech_ann['COMMODITY']=='CRP001') | (prod_tech_ann['COMMODITY']=='CRP002')].pivot_table(index='YEAR', 
                                                   columns='TECHNOLOGY',
                                                   values='VALUE', 
                                                   aggfunc='sum').reset_index().rename_axis(None, axis=1).rename(columns=det_col).fillna(0)
    tot_crop_prod=tot_crop_prod[tot_crop_prod['YEAR'].between(2020,2050)]

    #Electricity dataframes
    elec_gen= prod_tech_ann[(prod_tech_ann['TECHNOLOGY'].str.startswith('PWR')) & (prod_tech_ann['TECHNOLOGY'] !='PWRTRN') & (prod_tech_ann['COMMODITY'].str[:3] =='ELC')].pivot_table(index='YEAR', 
                                                   columns='TECHNOLOGY',
                                                   values='VALUE', 
                                                   aggfunc='sum').reset_index().rename_axis(None, axis=1).rename(columns=det_col).fillna(0)
    elec_gen=elec_gen[elec_gen['YEAR'].between(2020,2050)]
    pwr_cap= tot_ann_cap[tot_ann_cap['TECHNOLOGY'].isin(pwr_tech)].pivot_table(index='YEAR', 
                                                   columns='TECHNOLOGY',
                                                   values='VALUE', 
                                                   aggfunc='sum').reset_index().rename_axis(None, axis=1).rename(columns=det_col).fillna(0)
    pwr_cap=pwr_cap[pwr_cap['YEAR'].between(2020,2050)]
    # Energy supply by type (to be corrected)
    #energy_supply= prod_tech_ann[(prod_tech_ann['TECHNOLOGY'].str.startswith('PWR')) & (prod_tech_ann['TECHNOLOGY'] !='PWRTRN') & (prod_tech_ann['COMMODITY'].str[:3] =='ELC')].pivot_table(index='YEAR', 
                                                   #columns='TECHNOLOGY',
                                                   #values='VALUE', 
                                                   #aggfunc='sum').reset_index().rename_axis(None, axis=1).rename(columns=det_col).fillna(0)
    #energy_supply=energy_supply[energy_supply['YEAR'].between(2020,2050)]
    #Water dataframes
    wat_sup_req= useby_tech_ann[(useby_tech_ann['COMMODITY'].str.startswith('WAT001'))].pivot_table(index='YEAR', 
                                                   columns='TECHNOLOGY',
                                                   values='VALUE', 
                                                   aggfunc='sum').reset_index().rename_axis(None, axis=1).rename(columns=det_col).fillna(0)
    wat_sup_req=wat_sup_req[wat_sup_req['YEAR'].between(2020,2050)]
    wat_sup_cap= tot_ann_cap[tot_ann_cap['TECHNOLOGY'].isin(wat_tech)].pivot_table(index='YEAR', 
                                                   columns='TECHNOLOGY',
                                                   values='VALUE', 
                                                   aggfunc='sum').reset_index().rename_axis(None, axis=1).rename(columns=det_col).fillna(0)
    wat_sup_cap=wat_sup_cap[wat_sup_cap['YEAR'].between(2020,2050)]
    wat_sup_prod= prod_tech_ann[prod_tech_ann['TECHNOLOGY'].isin(wat_tech)].pivot_table(index='YEAR', 
                                                   columns='TECHNOLOGY',
                                                   values='VALUE', 
                                                   aggfunc='sum').reset_index().rename_axis(None, axis=1).rename(columns=det_col).fillna(0)
    wat_sup_prod=wat_sup_prod[wat_sup_prod['YEAR'].between(2020,2050)]
    #Emission/climate dataframes
    tot_emissions=emissions[emissions['EMISSION'] =='CO2EQ'].pivot_table(index='YEAR', 
                                                   columns='EMISSION',
                                                   values='VALUE', 
                                                   aggfunc='sum').reset_index().rename_axis(None, axis=1).rename(columns=det_col).fillna(0)
    tot_emissions=tot_emissions[tot_emissions['YEAR'].between(2020,2050)]
    tot_co2eq_emissions=emis_by_tech[emis_by_tech['EMISSION'] =='CO2EQ'].pivot_table(index='YEAR', 
                                                   columns='TECHNOLOGY',
                                                   values='VALUE', 
                                                   aggfunc='sum').reset_index().rename_axis(None, axis=1).rename(columns=det_col).fillna(0)
    tot_co2eq_emissions=tot_co2eq_emissions[tot_co2eq_emissions['YEAR'].between(2020,2050)]
    #Land plots (individual)
    df_plot(tot_land_cap,'Area in Thousand hectares','Area by land cover type'+"-"+scen)
    df_plot(tot_crop_cap,'Area in Thousand hectares','Total cropland area'+"-"+scen)
    df_plot(tot_crop_prod,'Production in Million Tonnes','Crop Production'+"-"+scen)
    #Energy plots (individual)
    df_plot(elec_gen,'Peta Joules','Electricity generation'+"-"+scen)
    df_plot(pwr_cap,'GigaWatts (GW)','Power Generation Capacity'+"-"+scen)
    #Water plots (individual)
    df_plot(wat_sup_req,'Billion Cubic Meters (BCM)','Water supply requirement'+"-"+scen)
    #df_plot(wat_sup_cap,'Billion Cubic Meters (BCM)','Water supply capacity')# removed as it was not necessary
    df_plot(wat_sup_prod,'Billion Cubic Meters (BCM)','Water supplied'+"-"+scen)
    #Climate plots (individual)
    df_plot(tot_emissions,'Carbon dioxide Equivalent(CO2-eq)','Annual emissions'+"-"+scen)
    df_plot(tot_co2eq_emissions,'Carbon dioxide Equivalent(CO2-eq)','Emissions by activity (CO2-eq)'+"-"+scen)
    ####*****###
    # Land Subplots
    fig=plt.figure(figsize=(10,5))
    ax1 = fig.add_subplot(211)
    ax2 = fig.add_subplot(223)
    ax3 = fig.add_subplot(224)
    #fig, axes = plt.subplots(nrows=2, ncols=2,figsize=(10,5))
    tot_land_cap.plot(x='YEAR',kind='bar',stacked=True, ax=ax1,legend=False,ylabel='Area-Thousand sq.km',color=[color_dict[x] for x in tot_land_cap.columns if x != 'YEAR']); ax1.set_title('Area by land cover type'+"-"+scen)
    tot_crop_cap.plot(x='YEAR',kind='bar',stacked=True, ax=ax3,legend=False,ylabel='Area-Thousand sq.km',color=[color_dict[x] for x in tot_crop_cap.columns if x != 'YEAR']); ax3.set_title('Total cropland area'+"-"+scen)
    tot_crop_prod.plot(x='YEAR',kind='bar',stacked=True, ax=ax2,legend=False,ylabel='Crop production- Million Tonnes',color=[color_dict[x] for x in tot_crop_cap.columns if x != 'YEAR']); ax2.set_title('Crop Production'+"-"+scen)
    ax1.set_xticks(range(0,40,10))
    ax1.set_ylim(0,16000)
    ax1.legend(bbox_to_anchor=(1.5,1),borderaxespad=0.2)
    ax2.set_xticks(range(0,40,10))
    ax2.set_ylim(0,13)
    ax3.set_xticks(range(0,40,10))
    ax3.set_ylim(0,650)
    #fig.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.) # used for group legend
    plt.tight_layout()
    #Hash the following line if you want to see inline graphs
    #plt.close()
    plt.savefig(homedir+"/land_subplots"+"-"+scen+".pdf")
    plt.savefig(homedir+"/land_subplots"+"-"+scen+".png")
    #************************#
    # Water Subplots
    fig=plt.figure(figsize=(10,3))
    ax1 = fig.add_subplot(121)
    ax2 = fig.add_subplot(122)
    #ax3 = fig.add_subplot(224)
    #fig, axes = plt.subplots(nrows=2, ncols=2,figsize=(10,5))
    wat_sup_req.plot(x='YEAR',kind='bar',stacked=True, ax=ax1,legend=False,ylabel='Billion cubic meters(BCM)',color=[color_dict[x] for x in wat_sup_req.columns if x != 'YEAR']); ax1.set_title('Water supply requirement'+"-"+scen)
    wat_sup_prod.plot(x='YEAR',kind='bar',stacked=True, ax=ax2,legend=False,ylabel='Billion cubic meters(BCM)',color=[color_dict[x] for x in wat_sup_prod.columns if x != 'YEAR']); ax2.set_title('Water Supplied'+"-"+scen)
    #wat_sup_cap.plot(x='YEAR',kind='bar',stacked=True, ax=ax3,legend=False,ylabel='Billion cubic meters(BCM)',color=[color_dict[x] for x in wat_sup_cap.columns if x != 'YEAR']); ax3.set_title('Water supply capacity'+"-"+scen)
    ax1.set_xticks(range(0,40,10))
    ax1.set_ylim(0,5)
    #ax1.legend(bbox_to_anchor=(1,1),borderaxespad=0.2)
    ax2.set_xticks(range(0,40,10))
    ax2.set_ylim(0,6)
    ax2.legend(bbox_to_anchor=(1.8,1))
    #ax3.set_xticks(range(0,40,10))
    #ax3.set_ylim(0,6)
    #ax3.legend(bbox_to_anchor=(1,1),borderaxespad=0.2)
    #fig.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.) # used for group legend
    plt.tight_layout()
    #Hash the following line if you want to see inline graphs
    #plt.close()
    plt.savefig(homedir+"/water_subplots"+"-"+scen+".pdf")
    plt.savefig(homedir+"/water_subplots"+"-"+scen+".png")
    ###***************##
    # Energy Subplots
    fig=plt.figure(figsize=(10,3))
    ax1 = fig.add_subplot(121)
    ax2 = fig.add_subplot(122)
    #fig, axes = plt.subplots(nrows=2, ncols=2,figsize=(10,5))
    elec_gen.plot(x='YEAR',kind='bar',stacked=True, ax=ax1,legend=False,ylabel='PetaJoules (PJ)',color=[color_dict[x] for x in elec_gen.columns if x != 'YEAR']); ax1.set_title('Electricity generation'+"-"+scen)
    pwr_cap.plot(x='YEAR',kind='bar',stacked=True, ax=ax2,legend=False,ylabel='GigaWatts (GW)',color=[color_dict[x] for x in pwr_cap.columns if x != 'YEAR']); ax2.set_title('Power Generation Capacity'+"-"+scen)
    ax1.set_xticks(range(0,40,10))
    ax1.set_ylim(0,500)
    #ax1.legend(bbox_to_anchor=(1,1),borderaxespad=0.2)
    ax2.set_xticks(range(0,40,10))
    ax2.set_ylim(0,30)
    ax2.legend(bbox_to_anchor=(1,1),borderaxespad=0.2)
    #fig.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.) # used for group legend
    plt.tight_layout()
    #Hash the following line if you want to see inline graphs
    #plt.close()
    plt.savefig(homedir+"/energy_subplots"+"-"+scen+".pdf")
    plt.savefig(homedir+"/energy_subplots"+"-"+scen+".png")
    #************###
    # Climate Subplots
    fig=plt.figure(figsize=(10,3))
    ax1 = fig.add_subplot(121)
    ax2 = fig.add_subplot(122)
    tot_emissions.plot(x='YEAR',kind='bar',stacked=True, ax=ax1,legend=False,ylabel='Carbon dioxide Equivalent(CO2-eq))',color=[color_dict[x] for x in tot_emissions.columns if x != 'YEAR']); ax1.set_title('Annual emissions'+"-"+scen)
    tot_co2eq_emissions.plot(x='YEAR',kind='bar',stacked=True, ax=ax2,legend=False,ylabel='Carbon dioxide Equivalent(CO2-eq)',color=[color_dict[x] for x in tot_co2eq_emissions.columns if x != 'YEAR']); ax2.set_title('Emissions by activity (CO2-eq)'+"-"+scen)
    ax1.set_xticks(range(0,40,10))
    ax1.set_ylim(-55,20)
    #ax1.legend(bbox_to_anchor=(1,1),borderaxespad=0.2)
    ax2.set_xticks(range(0,40,10))
    ax2.set_ylim(-80,40)
    #ax2.legend(bbox_to_anchor=(1,1),borderaxespad=0.2)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.) # used for group legend
    plt.tight_layout()
    #Hash the following line if you want to see inline graphs
    #plt.close()
    plt.savefig(homedir+"/climate_subplots"+"-"+scen+".pdf")
    plt.savefig(homedir+"/climate_subplots"+"-"+scen+".pdf")

In [ ]:
# Energy comparison
REF_prod_tech_ann=pd.read_csv('C:/git/teaching-CLEWs_visualization/results/REF'+'/ProductionByTechnologyAnnual.csv')
REF_prod_tech_ann.drop('REGION',axis=1,inplace=True)
REF_prod_tech_ann=REF_prod_tech_ann.set_index('YEAR')
WAT_prod_tech_ann=pd.read_csv('C:/git/teaching-CLEWs_visualization/results/WAT'+'/ProductionByTechnologyAnnual.csv')
WAT_prod_tech_ann.drop('REGION',axis=1,inplace=True)
WAT_prod_tech_ann=WAT_prod_tech_ann.set_index('YEAR')
CLM_prod_tech_ann=pd.read_csv('C:/git/teaching-CLEWs_visualization/results/CLM'+'/ProductionByTechnologyAnnual.csv')
CLM_prod_tech_ann.drop('REGION',axis=1,inplace=True)
CLM_prod_tech_ann=CLM_prod_tech_ann.set_index('YEAR')
LND_prod_tech_ann=pd.read_csv('C:/git/teaching-CLEWs_visualization/results/LND'+'/ProductionByTechnologyAnnual.csv')
LND_prod_tech_ann.drop('REGION',axis=1,inplace=True)
LND_prod_tech_ann=LND_prod_tech_ann.set_index('YEAR')
COM_prod_tech_ann=pd.read_csv('C:/git/teaching-CLEWs_visualization/results/COM'+'/ProductionByTechnologyAnnual.csv')
COM_prod_tech_ann.drop('REGION',axis=1,inplace=True)
COM_prod_tech_ann=COM_prod_tech_ann.set_index('YEAR')
ENR_prod_tech_ann=pd.read_csv('C:/git/teaching-CLEWs_visualization/results/ENR'+'/ProductionByTechnologyAnnual.csv')
ENR_prod_tech_ann.drop('REGION',axis=1,inplace=True)
ENR_prod_tech_ann=ENR_prod_tech_ann.set_index('YEAR')
## dataframe management
ENR_elec_gen=ENR_prod_tech_ann[(ENR_prod_tech_ann['TECHNOLOGY'].str.startswith('PWR')) & (ENR_prod_tech_ann['TECHNOLOGY'] !='PWRTRN') & (ENR_prod_tech_ann['COMMODITY'].str[:3] =='ELC')].pivot_table(index='YEAR', 
                                                   columns='TECHNOLOGY',
                                                   values='VALUE', 
                                                   aggfunc='sum').reset_index().rename_axis(None, axis=1).rename(columns=det_col).fillna(0)
ENR_elec_gen=ENR_elec_gen[ENR_elec_gen['YEAR'].between(2020,2050)]
#df_plot(ENR_elec_gen,'Peta Joules','Electricity generation')
REF_elec_gen=REF_prod_tech_ann[(REF_prod_tech_ann['TECHNOLOGY'].str.startswith('PWR')) & (REF_prod_tech_ann['TECHNOLOGY'] !='PWRTRN') & (REF_prod_tech_ann['COMMODITY'].str[:3] =='ELC')].pivot_table(index='YEAR', 
                                                   columns='TECHNOLOGY',
                                                   values='VALUE', 
                                                   aggfunc='sum').reset_index().rename_axis(None, axis=1).rename(columns=det_col).fillna(0)
REF_elec_gen=REF_elec_gen[REF_elec_gen['YEAR'].between(2020,2050)]
#df_plot(REF_elec_gen,'Peta Joules','Electricity generation')
LND_elec_gen=LND_prod_tech_ann[(LND_prod_tech_ann['TECHNOLOGY'].str.startswith('PWR')) & (LND_prod_tech_ann['TECHNOLOGY'] !='PWRTRN') & (LND_prod_tech_ann['COMMODITY'].str[:3] =='ELC')].pivot_table(index='YEAR', 
                                                   columns='TECHNOLOGY',
                                                   values='VALUE', 
                                                   aggfunc='sum').reset_index().rename_axis(None, axis=1).rename(columns=det_col).fillna(0)
LND_elec_gen=LND_elec_gen[LND_elec_gen['YEAR'].between(2020,2050)]
#df_plot(LND_elec_gen,'Peta Joules','Electricity generation')
COM_elec_gen=COM_prod_tech_ann[(COM_prod_tech_ann['TECHNOLOGY'].str.startswith('PWR')) & (COM_prod_tech_ann['TECHNOLOGY'] !='PWRTRN') & (COM_prod_tech_ann['COMMODITY'].str[:3] =='ELC')].pivot_table(index='YEAR', 
                                                   columns='TECHNOLOGY',
                                                   values='VALUE', 
                                                   aggfunc='sum').reset_index().rename_axis(None, axis=1).rename(columns=det_col).fillna(0)
COM_elec_gen=COM_elec_gen[COM_elec_gen['YEAR'].between(2020,2050)]
#df_plot(COMBO_elec_gen,'Peta Joules','Electricity generation')
WAT_elec_gen=WAT_prod_tech_ann[(WAT_prod_tech_ann['TECHNOLOGY'].str.startswith('PWR')) & (WAT_prod_tech_ann['TECHNOLOGY'] !='PWRTRN') & (WAT_prod_tech_ann['COMMODITY'].str[:3] =='ELC')].pivot_table(index='YEAR', 
                                                   columns='TECHNOLOGY',
                                                   values='VALUE', 
                                                   aggfunc='sum').reset_index().rename_axis(None, axis=1).rename(columns=det_col).fillna(0)
WAT_elec_gen=WAT_elec_gen[WAT_elec_gen['YEAR'].between(2020,2050)]
#df_plot(WAT_elec_gen,'Peta Joules','Electricity generation')
CLM_elec_gen=CLM_prod_tech_ann[(CLM_prod_tech_ann['TECHNOLOGY'].str.startswith('PWR')) & (CLM_prod_tech_ann['TECHNOLOGY'] !='PWRTRN') & (CLM_prod_tech_ann['COMMODITY'].str[:3] =='ELC')].pivot_table(index='YEAR', 
                                                   columns='TECHNOLOGY',
                                                   values='VALUE', 
                                                   aggfunc='sum').reset_index().rename_axis(None, axis=1).rename(columns=det_col).fillna(0)
CLM_elec_gen=CLM_elec_gen[CLM_elec_gen['YEAR'].between(2020,2050)]
#df_plot(CLM_elec_gen,'Peta Joules','Electricity generation')
#Scenario subtraction
ENR_REF_elec_gen=ENR_elec_gen.set_index('YEAR')-REF_elec_gen.set_index('YEAR')
#df_plot_rl(ENR_REF_elec_gen,'Peta Joules (PJ)','Electricity generation'+ ' (ENR_REF)')
ENR_REF_elec_gen=ENR_REF_elec_gen.reset_index()
LND_REF_elec_gen=LND_elec_gen.set_index('YEAR')-REF_elec_gen.set_index('YEAR')
#df_plot_rl(LND_REF_elec_gen,'Peta Joules (PJ)','Electricity generation'+ ' (LND_REF)')
LND_REF_elec_gen=LND_REF_elec_gen.reset_index().copy()
WAT_REF_elec_gen=WAT_elec_gen.set_index('YEAR')-REF_elec_gen.set_index('YEAR')
#df_plot_rl(WAT_REF_elec_gen,'Peta Joules (PJ)','Electricity generation'+ ' (WAT_REF)')
WAT_REF_elec_gen=WAT_REF_elec_gen.reset_index().copy()
COM_REF_elec_gen=COM_elec_gen.set_index('YEAR')-REF_elec_gen.set_index('YEAR')
#df_plot_rl(COMBO_REF_elec_gen,'Peta Joules (PJ)','Electricity generation'+ ' (COMBO_REF)')
COM_REF_elec_gen=COM_REF_elec_gen.reset_index().copy()
CLM_REF_elec_gen=CLM_elec_gen.set_index('YEAR')-REF_elec_gen.set_index('YEAR')
#df_plot_rl(CLM_REF_elec_gen,'Peta Joules (PJ)','Electricity generation'+ ' (CLM_REF)')
CLM_REF_elec_gen=CLM_REF_elec_gen.reset_index().copy()
#
# Energy Subplots
fig=plt.figure(figsize=(20,5))
ax1 = fig.add_subplot(151)
ax2 = fig.add_subplot(152)
ax3 = fig.add_subplot(153)
ax4 = fig.add_subplot(154)
ax5 = fig.add_subplot(155)
ENR_REF_elec_gen.plot(x='YEAR',kind='bar',stacked=True, ax=ax1,legend=False,ylabel='PetaJoules (PJ)',color=[color_dict[x] for x in elec_gen.columns if x != 'YEAR']); ax1.set_title('ENR-REF')
WAT_REF_elec_gen.plot(x='YEAR',kind='bar',stacked=True, ax=ax2,legend=False,color=[color_dict[x] for x in elec_gen.columns if x != 'YEAR']); ax2.set_title('WAT-REF')
LND_REF_elec_gen.plot(x='YEAR',kind='bar',stacked=True, ax=ax3,legend=False,color=[color_dict[x] for x in elec_gen.columns if x != 'YEAR']); ax3.set_title('LND-REF')
CLM_REF_elec_gen.plot(x='YEAR',kind='bar',stacked=True, ax=ax4,legend=False,color=[color_dict[x] for x in elec_gen.columns if x != 'YEAR']); ax4.set_title('CLM-REF')
COM_REF_elec_gen.plot(x='YEAR',kind='bar',stacked=True, ax=ax5,legend=False,color=[color_dict[x] for x in elec_gen.columns if x != 'YEAR']); ax5.set_title('COM-REF')
ax1.set_xticks(range(0,40,10))
#ax1.set_ylim(0,500)
ax2.set_xticks(range(0,40,10))
#ax2.set_ylim(0,500)
ax3.set_xticks(range(0,40,10))
#ax3.set_ylim(0,500)
ax4.set_xticks(range(0,40,10))
#ax4.set_ylim(0,500)
ax5.set_xticks(range(0,40,10))
#ax5.set_ylim(0,500)
ax5.legend(bbox_to_anchor=(1,1),borderaxespad=0.2)
#fig.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.) # used for group legend
plt.suptitle('Electricity Generation')
plt.tight_layout()
plt.savefig("C:/git/teaching-CLEWs_visualization/scenario_graphs/"+"energy_subplots_comp.pdf")
plt.savefig("C:/git/teaching-CLEWs_visualization/scenario_graphs/"+"energy_subplots_comp.png")

In [ ]:
# Climate comparison
REF_emis_by_tech=pd.read_csv('C:/git/teaching-CLEWs_visualization/results/REF'+'/AnnualTechnologyEmission.csv')
REF_emis_by_tech.drop('REGION',axis=1,inplace=True)
REF_emis_by_tech=REF_emis_by_tech.set_index('YEAR')
WAT_emis_by_tech=pd.read_csv('C:/git/teaching-CLEWs_visualization/results/WAT'+'/AnnualTechnologyEmission.csv')
WAT_emis_by_tech.drop('REGION',axis=1,inplace=True)
WAT_emis_by_tech=WAT_emis_by_tech.set_index('YEAR')
CLM_emis_by_tech=pd.read_csv('C:/git/teaching-CLEWs_visualization/results/CLM'+'/AnnualTechnologyEmission.csv')
CLM_emis_by_tech.drop('REGION',axis=1,inplace=True)
CLM_emis_by_tech=CLM_emis_by_tech.set_index('YEAR')
LND_emis_by_tech=pd.read_csv('C:/git/teaching-CLEWs_visualization/results/LND'+'/AnnualTechnologyEmission.csv')
LND_emis_by_tech.drop('REGION',axis=1,inplace=True)
LND_emis_by_tech=LND_emis_by_tech.set_index('YEAR')
COM_emis_by_tech=pd.read_csv('C:/git/teaching-CLEWs_visualization/results/COM'+'/AnnualTechnologyEmission.csv')
COM_emis_by_tech.drop('REGION',axis=1,inplace=True)
COM_emis_by_tech=COM_emis_by_tech.set_index('YEAR')
ENR_emis_by_tech=pd.read_csv('C:/git/teaching-CLEWs_visualization/results/ENR'+'/AnnualTechnologyEmission.csv')
ENR_emis_by_tech.drop('REGION',axis=1,inplace=True)
ENR_emis_by_tech=ENR_emis_by_tech.set_index('YEAR')
#
REF_co2eq_emissions=REF_emis_by_tech[REF_emis_by_tech['EMISSION'] =='CO2EQ'].pivot_table(index='YEAR', 
                                               columns='TECHNOLOGY',
                                               values='VALUE', 
                                               aggfunc='sum').reset_index().rename_axis(None, axis=1).rename(columns=det_col).fillna(0)
REF_co2eq_emissions=REF_co2eq_emissions[REF_co2eq_emissions['YEAR'].between(2020,2050)]
ENR_co2eq_emissions=ENR_emis_by_tech[ENR_emis_by_tech['EMISSION'] =='CO2EQ'].pivot_table(index='YEAR', 
                                               columns='TECHNOLOGY',
                                               values='VALUE', 
                                               aggfunc='sum').reset_index().rename_axis(None, axis=1).rename(columns=det_col).fillna(0)
ENR_co2eq_emissions=ENR_co2eq_emissions[ENR_co2eq_emissions['YEAR'].between(2020,2050)]
WAT_co2eq_emissions=WAT_emis_by_tech[WAT_emis_by_tech['EMISSION'] =='CO2EQ'].pivot_table(index='YEAR', 
                                               columns='TECHNOLOGY',
                                               values='VALUE', 
                                               aggfunc='sum').reset_index().rename_axis(None, axis=1).rename(columns=det_col).fillna(0)
WAT_co2eq_emissions=WAT_co2eq_emissions[WAT_co2eq_emissions['YEAR'].between(2020,2050)]
LND_co2eq_emissions=LND_emis_by_tech[LND_emis_by_tech['EMISSION'] =='CO2EQ'].pivot_table(index='YEAR', 
                                               columns='TECHNOLOGY',
                                               values='VALUE', 
                                               aggfunc='sum').reset_index().rename_axis(None, axis=1).rename(columns=det_col).fillna(0)
LND_co2eq_emissions=LND_co2eq_emissions[LND_co2eq_emissions['YEAR'].between(2020,2050)]
CLM_co2eq_emissions=CLM_emis_by_tech[CLM_emis_by_tech['EMISSION'] =='CO2EQ'].pivot_table(index='YEAR', 
                                               columns='TECHNOLOGY',
                                               values='VALUE', 
                                               aggfunc='sum').reset_index().rename_axis(None, axis=1).rename(columns=det_col).fillna(0)
CLM_co2eq_emissions=CLM_co2eq_emissions[CLM_co2eq_emissions['YEAR'].between(2020,2050)]
COM_co2eq_emissions=COM_emis_by_tech[COM_emis_by_tech['EMISSION'] =='CO2EQ'].pivot_table(index='YEAR', 
                                               columns='TECHNOLOGY',
                                               values='VALUE', 
                                               aggfunc='sum').reset_index().rename_axis(None, axis=1).rename(columns=det_col).fillna(0)
COM_co2eq_emissions=COM_co2eq_emissions[COM_co2eq_emissions['YEAR'].between(2020,2050)]

#
ENR_REF_co2eq_emissions=ENR_co2eq_emissions.set_index('YEAR')-REF_co2eq_emissions.set_index('YEAR')
#df_plot_rl(ENR_REF_co2eq_emissions,'Peta Joules (PJ)','Electricity generation'+ ' (ENR_REF)')
ENR_REF_co2eq_emissions=ENR_REF_co2eq_emissions.reset_index()
LND_REF_co2eq_emissions=LND_co2eq_emissions.set_index('YEAR')-REF_co2eq_emissions.set_index('YEAR')
#df_plot_rl(LND_REF_co2eq_emissions,'Peta Joules (PJ)','Electricity generation'+ ' (LND_REF)')
LND_REF_co2eq_emissions=LND_REF_co2eq_emissions.reset_index().copy()
WAT_REF_co2eq_emissions=WAT_co2eq_emissions.set_index('YEAR')-REF_co2eq_emissions.set_index('YEAR')
#df_plot_rl(WAT_REF_co2eq_emissions,'Peta Joules (PJ)','Electricity generation'+ ' (WAT_REF)')
WAT_REF_co2eq_emissions=WAT_REF_co2eq_emissions.reset_index().copy()
COM_REF_co2eq_emissions=COM_co2eq_emissions.set_index('YEAR')-REF_co2eq_emissions.set_index('YEAR')
#df_plot_rl(COMBO_REF_co2eq_emissions,'Peta Joules (PJ)','Electricity generation'+ ' (COMBO_REF)')
COM_REF_co2eq_emissions=COM_REF_co2eq_emissions.reset_index().copy()
CLM_REF_co2eq_emissions=CLM_co2eq_emissions.set_index('YEAR')-REF_co2eq_emissions.set_index('YEAR')
#df_plot_rl(CLM_REF_co2eq_emissions,'Peta Joules (PJ)','Electricity generation'+ ' (CLM_REF)')
CLM_REF_co2eq_emissions=CLM_REF_co2eq_emissions.reset_index().copy()
#
# climate Subplots
fig=plt.figure(figsize=(20,5))
ax1 = fig.add_subplot(151)
ax2 = fig.add_subplot(152)
ax3 = fig.add_subplot(153)
ax4 = fig.add_subplot(154)
ax5 = fig.add_subplot(155)
ENR_REF_co2eq_emissions.plot(x='YEAR',kind='bar',stacked=True, ax=ax1,legend=False,ylabel='Carbon dioxide Equivalent(CO2-eq)',color=[color_dict[x] for x in tot_co2eq_emissions.columns if x != 'YEAR']); ax1.set_title('ENR-REF')
WAT_REF_co2eq_emissions.plot(x='YEAR',kind='bar',stacked=True, ax=ax2,legend=False,color=[color_dict[x] for x in tot_co2eq_emissions.columns if x != 'YEAR']); ax2.set_title('WAT-REF')
LND_REF_co2eq_emissions.plot(x='YEAR',kind='bar',stacked=True, ax=ax3,legend=False,color=[color_dict[x] for x in tot_co2eq_emissions.columns if x != 'YEAR']); ax3.set_title('LND-REF')
CLM_REF_co2eq_emissions.plot(x='YEAR',kind='bar',stacked=True, ax=ax4,legend=False,color=[color_dict[x] for x in tot_co2eq_emissions.columns if x != 'YEAR']); ax4.set_title('CLM-REF')
COM_REF_co2eq_emissions.plot(x='YEAR',kind='bar',stacked=True, ax=ax5,legend=False,color=[color_dict[x] for x in tot_co2eq_emissions.columns if x != 'YEAR']); ax5.set_title('COM-REF')
ax1.set_xticks(range(0,40,10))
#ax1.set_ylim(0,500)
ax2.set_xticks(range(0,40,10))
#ax2.set_ylim(0,500)
ax3.set_xticks(range(0,40,10))
#ax3.set_ylim(0,500)
ax4.set_xticks(range(0,40,10))
#ax4.set_ylim(0,500)
ax5.set_xticks(range(0,40,10))
#ax5.set_ylim(0,500)
ax5.legend(bbox_to_anchor=(1,1),borderaxespad=0.2)
#fig.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.) # used for group legend
plt.suptitle('Emissions by activity (CO2-eq)')
plt.tight_layout()
plt.savefig("C:/git/teaching-CLEWs_visualization/scenario_graphs/"+"climate_subplots_comp.pdf")
plt.savefig("C:/git/teaching-CLEWs_visualization/scenario_graphs/"+"climate_subplots_comp.png")

In [ ]:
#Water comaprison
REF_useby_tech_ann=pd.read_csv('C:/git/teaching-CLEWs_visualization/results/REF'+'/UseByTechnology.csv')
REF_useby_tech_ann.drop('REGION',axis=1,inplace=True)
REF_useby_tech_ann=REF_useby_tech_ann.set_index('YEAR')
WAT_useby_tech_ann=pd.read_csv('C:/git/teaching-CLEWs_visualization/results/WAT'+'/UseByTechnology.csv')
WAT_useby_tech_ann.drop('REGION',axis=1,inplace=True)
WAT_useby_tech_ann=WAT_useby_tech_ann.set_index('YEAR')
CLM_useby_tech_ann=pd.read_csv('C:/git/teaching-CLEWs_visualization/results/CLM'+'/UseByTechnology.csv')
CLM_useby_tech_ann.drop('REGION',axis=1,inplace=True)
CLM_useby_tech_ann=CLM_useby_tech_ann.set_index('YEAR')
LND_useby_tech_ann=pd.read_csv('C:/git/teaching-CLEWs_visualization/results/LND'+'/UseByTechnology.csv')
LND_useby_tech_ann.drop('REGION',axis=1,inplace=True)
LND_useby_tech_ann=LND_useby_tech_ann.set_index('YEAR')
COM_useby_tech_ann=pd.read_csv('C:/git/teaching-CLEWs_visualization/results/COM'+'/UseByTechnology.csv')
COM_useby_tech_ann.drop('REGION',axis=1,inplace=True)
COM_useby_tech_ann=COM_useby_tech_ann.set_index('YEAR')
ENR_useby_tech_ann=pd.read_csv('C:/git/teaching-CLEWs_visualization/results/ENR'+'/UseByTechnology.csv')
ENR_useby_tech_ann.drop('REGION',axis=1,inplace=True)
ENR_useby_tech_ann=ENR_useby_tech_ann.set_index('YEAR')
#
REF_wat_sup_req= REF_useby_tech_ann[(REF_useby_tech_ann['COMMODITY'].str.startswith('WAT001'))].pivot_table(index='YEAR', 
                                               columns='TECHNOLOGY',
                                               values='VALUE', 
                                               aggfunc='sum').reset_index().rename_axis(None, axis=1).rename(columns=det_col).fillna(0)
REF_wat_sup_req=REF_wat_sup_req[REF_wat_sup_req['YEAR'].between(2020,2050)]
WAT_wat_sup_req= WAT_useby_tech_ann[(WAT_useby_tech_ann['COMMODITY'].str.startswith('WAT001'))].pivot_table(index='YEAR', 
                                               columns='TECHNOLOGY',
                                               values='VALUE', 
                                               aggfunc='sum').reset_index().rename_axis(None, axis=1).rename(columns=det_col).fillna(0)
WAT_wat_sup_req=WAT_wat_sup_req[WAT_wat_sup_req['YEAR'].between(2020,2050)]
LND_wat_sup_req= LND_useby_tech_ann[(LND_useby_tech_ann['COMMODITY'].str.startswith('WAT001'))].pivot_table(index='YEAR', 
                                               columns='TECHNOLOGY',
                                               values='VALUE', 
                                               aggfunc='sum').reset_index().rename_axis(None, axis=1).rename(columns=det_col).fillna(0)
LND_wat_sup_req=LND_wat_sup_req[LND_wat_sup_req['YEAR'].between(2020,2050)]
CLM_wat_sup_req= CLM_useby_tech_ann[(CLM_useby_tech_ann['COMMODITY'].str.startswith('WAT001'))].pivot_table(index='YEAR', 
                                               columns='TECHNOLOGY',
                                               values='VALUE', 
                                               aggfunc='sum').reset_index().rename_axis(None, axis=1).rename(columns=det_col).fillna(0)
CLM_wat_sup_req=CLM_wat_sup_req[CLM_wat_sup_req['YEAR'].between(2020,2050)]
ENR_wat_sup_req= ENR_useby_tech_ann[(ENR_useby_tech_ann['COMMODITY'].str.startswith('WAT001'))].pivot_table(index='YEAR', 
                                               columns='TECHNOLOGY',
                                               values='VALUE', 
                                               aggfunc='sum').reset_index().rename_axis(None, axis=1).rename(columns=det_col).fillna(0)
ENR_wat_sup_req=ENR_wat_sup_req[ENR_wat_sup_req['YEAR'].between(2020,2050)]
COM_wat_sup_req= COM_useby_tech_ann[(COM_useby_tech_ann['COMMODITY'].str.startswith('WAT001'))].pivot_table(index='YEAR', 
                                               columns='TECHNOLOGY',
                                               values='VALUE', 
                                               aggfunc='sum').reset_index().rename_axis(None, axis=1).rename(columns=det_col).fillna(0)
COM_wat_sup_req=COM_wat_sup_req[COM_wat_sup_req['YEAR'].between(2020,2050)]


#
ENR_REF_wat_sup_req=ENR_wat_sup_req.set_index('YEAR')-REF_wat_sup_req.set_index('YEAR')
#df_plot_rl(ENR_REF_wat_sup_req,'Peta Joules (PJ)','Electricity generation'+ ' (ENR_REF)')
ENR_REF_wat_sup_req=ENR_REF_wat_sup_req.reset_index()
LND_REF_wat_sup_req=LND_wat_sup_req.set_index('YEAR')-REF_wat_sup_req.set_index('YEAR')
#df_plot_rl(LND_REF_wat_sup_req,'Peta Joules (PJ)','Electricity generation'+ ' (LND_REF)')
LND_REF_wat_sup_req=LND_REF_wat_sup_req.reset_index().copy()
WAT_REF_wat_sup_req=WAT_wat_sup_req.set_index('YEAR')-REF_wat_sup_req.set_index('YEAR')
#df_plot_rl(WAT_REF_wat_sup_req,'Peta Joules (PJ)','Electricity generation'+ ' (WAT_REF)')
WAT_REF_wat_sup_req=WAT_REF_wat_sup_req.reset_index().copy()
COM_REF_wat_sup_req=COM_wat_sup_req.set_index('YEAR')-REF_wat_sup_req.set_index('YEAR')
#df_plot_rl(COMBO_REF_wat_sup_req,'Peta Joules (PJ)','Electricity generation'+ ' (COMBO_REF)')
COM_REF_wat_sup_req=COM_REF_wat_sup_req.reset_index().copy()
CLM_REF_wat_sup_req=CLM_wat_sup_req.set_index('YEAR')-REF_wat_sup_req.set_index('YEAR')
#df_plot_rl(CLM_REF_wat_sup_req,'Peta Joules (PJ)','Electricity generation'+ ' (CLM_REF)')
CLM_REF_wat_sup_req=CLM_REF_wat_sup_req.reset_index().copy()
#
# Water Subplots
fig=plt.figure(figsize=(20,5))
ax1 = fig.add_subplot(151)
ax2 = fig.add_subplot(152)
ax3 = fig.add_subplot(153)
ax4 = fig.add_subplot(154)
ax5 = fig.add_subplot(155)
ENR_REF_wat_sup_req.plot(x='YEAR',kind='bar',stacked=True, ax=ax1,legend=False,ylabel='Billion cubic meters (BCM)',color=[color_dict[x] for x in wat_sup_req.columns if x != 'YEAR']); ax1.set_title('ENR-REF')
WAT_REF_wat_sup_req.plot(x='YEAR',kind='bar',stacked=True, ax=ax2,legend=False,color=[color_dict[x] for x in wat_sup_req.columns if x != 'YEAR']); ax2.set_title('WAT-REF')
LND_REF_wat_sup_req.plot(x='YEAR',kind='bar',stacked=True, ax=ax3,legend=False,color=[color_dict[x] for x in wat_sup_req.columns if x != 'YEAR']); ax3.set_title('LND-REF')
CLM_REF_wat_sup_req.plot(x='YEAR',kind='bar',stacked=True, ax=ax4,legend=False,color=[color_dict[x] for x in wat_sup_req.columns if x != 'YEAR']); ax4.set_title('CLM-REF')
COM_REF_wat_sup_req.plot(x='YEAR',kind='bar',stacked=True, ax=ax5,legend=False,color=[color_dict[x] for x in wat_sup_req.columns if x != 'YEAR']); ax5.set_title('COM-REF')
ax1.set_xticks(range(0,40,10))
#ax1.set_ylim(0,500)
ax2.set_xticks(range(0,40,10))
#ax2.set_ylim(0,500)
ax3.set_xticks(range(0,40,10))
#ax3.set_ylim(0,500)
ax4.set_xticks(range(0,40,10))
#ax4.set_ylim(0,500)
ax5.set_xticks(range(0,40,10))
#ax5.set_ylim(0,500)
ax5.legend(bbox_to_anchor=(1,1),borderaxespad=0.2)
#fig.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.) # used for group legend
plt.suptitle('Water supply requirement')
plt.tight_layout()
plt.savefig("C:/git/teaching-CLEWs_visualization/scenario_graphs/"+"water_subplots_comp.pdf")
plt.savefig("C:/git/teaching-CLEWs_visualization/scenario_graphs/"+"water_subplots_comp.png")


In [ ]:
#Land comparison
REF_tot_land_cap= REF_prod_tech_ann[REF_prod_tech_ann['COMMODITY']=='LND001'].pivot_table(index='YEAR', 
                                               columns='TECHNOLOGY',
                                               values='VALUE', 
                                               aggfunc='sum').reset_index().rename_axis(None, axis=1).rename(columns=det_col).fillna(0)
REF_tot_land_cap=REF_tot_land_cap[REF_tot_land_cap['YEAR'].between(2020,2050)]
WAT_tot_land_cap= WAT_prod_tech_ann[WAT_prod_tech_ann['COMMODITY']=='LND001'].pivot_table(index='YEAR', 
                                               columns='TECHNOLOGY',
                                               values='VALUE', 
                                               aggfunc='sum').reset_index().rename_axis(None, axis=1).rename(columns=det_col).fillna(0)
WAT_tot_land_cap=WAT_tot_land_cap[WAT_tot_land_cap['YEAR'].between(2020,2050)]
LND_tot_land_cap= LND_prod_tech_ann[LND_prod_tech_ann['COMMODITY']=='LND001'].pivot_table(index='YEAR', 
                                               columns='TECHNOLOGY',
                                               values='VALUE', 
                                               aggfunc='sum').reset_index().rename_axis(None, axis=1).rename(columns=det_col).fillna(0)
LND_tot_land_cap=LND_tot_land_cap[LND_tot_land_cap['YEAR'].between(2020,2050)]
ENR_tot_land_cap= ENR_prod_tech_ann[ENR_prod_tech_ann['COMMODITY']=='LND001'].pivot_table(index='YEAR', 
                                               columns='TECHNOLOGY',
                                               values='VALUE', 
                                               aggfunc='sum').reset_index().rename_axis(None, axis=1).rename(columns=det_col).fillna(0)
ENR_tot_land_cap=ENR_tot_land_cap[ENR_tot_land_cap['YEAR'].between(2020,2050)]
CLM_tot_land_cap= CLM_prod_tech_ann[CLM_prod_tech_ann['COMMODITY']=='LND001'].pivot_table(index='YEAR', 
                                               columns='TECHNOLOGY',
                                               values='VALUE', 
                                               aggfunc='sum').reset_index().rename_axis(None, axis=1).rename(columns=det_col).fillna(0)
CLM_tot_land_cap=CLM_tot_land_cap[CLM_tot_land_cap['YEAR'].between(2020,2050)]
COM_tot_land_cap= COM_prod_tech_ann[COM_prod_tech_ann['COMMODITY']=='LND001'].pivot_table(index='YEAR', 
                                               columns='TECHNOLOGY',
                                               values='VALUE', 
                                               aggfunc='sum').reset_index().rename_axis(None, axis=1).rename(columns=det_col).fillna(0)
COM_tot_land_cap=COM_tot_land_cap[COM_tot_land_cap['YEAR'].between(2020,2050)]

#
ENR_REF_tot_land_cap=ENR_tot_land_cap.set_index('YEAR')-REF_tot_land_cap.set_index('YEAR')
#df_plot_rl(ENR_REF_tot_land_cap,'Peta Joules (PJ)','Electricity generation'+ ' (ENR_REF)')
ENR_REF_tot_land_cap=ENR_REF_tot_land_cap.reset_index()
LND_REF_tot_land_cap=LND_tot_land_cap.set_index('YEAR')-REF_tot_land_cap.set_index('YEAR')
#df_plot_rl(LND_REF_tot_land_cap,'Peta Joules (PJ)','Electricity generation'+ ' (LND_REF)')
LND_REF_tot_land_cap=LND_REF_tot_land_cap.reset_index().copy()
WAT_REF_tot_land_cap=WAT_tot_land_cap.set_index('YEAR')-REF_tot_land_cap.set_index('YEAR')
#df_plot_rl(WAT_REF_tot_land_cap,'Peta Joules (PJ)','Electricity generation'+ ' (WAT_REF)')
WAT_REF_tot_land_cap=WAT_REF_tot_land_cap.reset_index().copy()
COM_REF_tot_land_cap=COM_tot_land_cap.set_index('YEAR')-REF_tot_land_cap.set_index('YEAR')
#df_plot_rl(COMBO_REF_tot_land_cap,'Peta Joules (PJ)','Electricity generation'+ ' (COMBO_REF)')
COM_REF_tot_land_cap=COM_REF_tot_land_cap.reset_index().copy()
CLM_REF_tot_land_cap=CLM_tot_land_cap.set_index('YEAR')-REF_tot_land_cap.set_index('YEAR')
#df_plot_rl(CLM_REF_tot_land_cap,'Peta Joules (PJ)','Electricity generation'+ ' (CLM_REF)')
CLM_REF_tot_land_cap=CLM_REF_tot_land_cap.reset_index().copy()
#
# Land Subplots
fig=plt.figure(figsize=(20,5))
ax1 = fig.add_subplot(151)
ax2 = fig.add_subplot(152)
ax3 = fig.add_subplot(153)
ax4 = fig.add_subplot(154)
ax5 = fig.add_subplot(155)
ENR_REF_tot_land_cap.plot(x='YEAR',kind='bar',stacked=True, ax=ax1,legend=False,ylabel='Area-Thousand sq.km',color=[color_dict[x] for x in tot_land_cap.columns if x != 'YEAR']); ax1.set_title('ENR-REF')
WAT_REF_tot_land_cap.plot(x='YEAR',kind='bar',stacked=True, ax=ax2,legend=False,color=[color_dict[x] for x in tot_land_cap.columns if x != 'YEAR']); ax2.set_title('WAT-REF')
LND_REF_tot_land_cap.plot(x='YEAR',kind='bar',stacked=True, ax=ax3,legend=False,color=[color_dict[x] for x in tot_land_cap.columns if x != 'YEAR']); ax3.set_title('LND-REF')
CLM_REF_tot_land_cap.plot(x='YEAR',kind='bar',stacked=True, ax=ax4,legend=False,color=[color_dict[x] for x in tot_land_cap.columns if x != 'YEAR']); ax4.set_title('CLM-REF')
COM_REF_tot_land_cap.plot(x='YEAR',kind='bar',stacked=True, ax=ax5,legend=False,color=[color_dict[x] for x in tot_land_cap.columns if x != 'YEAR']); ax5.set_title('COM-REF')
ax1.set_xticks(range(0,40,10))
#ax1.set_ylim(0,500)
ax2.set_xticks(range(0,40,10))
#ax2.set_ylim(0,500)
ax3.set_xticks(range(0,40,10))
#ax3.set_ylim(0,500)
ax4.set_xticks(range(0,40,10))
#ax4.set_ylim(0,500)
ax5.set_xticks(range(0,40,10))
#ax5.set_ylim(0,500)
ax5.legend(bbox_to_anchor=(1,1),borderaxespad=0.2)
#fig.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.) # used for group legend
plt.suptitle('Area by land cover type')
plt.tight_layout()
plt.savefig("C:/git/teaching-CLEWs_visualization/scenario_graphs/"+"land_subplots_comp.pdf")
plt.savefig("C:/git/teaching-CLEWs_visualization/scenario_graphs/"+"land_subplots_comp.png")